# Wind power forecasting in the Amprion TSO zone

This is a capstone project from the course series [Advanced Data Science Specialization](https://www.coursera.org/specializations/advanced-data-science-ibm). It is a small example for show-casing what was learned in the course.

- Goal: Forecast next quarter-hourly value based on the past 24 values
- Data source: [Actual wind generation](https://www.amprion.net/Netzkennzahlen/Windenergieeinspeisung/) in the TSO zone
- Feature engineering: prepare historic data and datetime information
- Model: Single-layer LSTM model

**Usage**

You might have to install the requirements from here: [https://raw.githubusercontent.com/gplssm/Wind_power_forecasting_LSTM/main/requirements.txt](https://raw.githubusercontent.com/gplssm/Wind_power_forecasting_LSTM/main/requirements.txt)

Use the first 3 code cells for parametrizing data preparation and model definition and training.
For example, switch `train_model` to False for loading an already trained model from file.

**Define data range for testing and training**

Start and end day are both included

In [ ]:
start_day = "2019-01-01"
end_day = "2020-12-31"

**Define parameters for data pre-processing and feature engineering**

In [ ]:
timesteps_past = 24
timesteps_future = 1
stride = 1
batch_size = 64
test_fraction = 0.2 # possible range 0..1

**Parameters for model definition**

In [ ]:
train_model = True
stateful = False
lstm_nodes = 144
epochs = 10

In [ ]:
filename_model = f"wind_forecasting_{'stateful' if stateful else 'stateless'}_LSTM-{lstm_nodes}-nodes_{epochs}-epochs_adam_mae.keras"
print(filename_model)

In [ ]:
import datetime
import keras
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import seaborn as sns
from urllib.request import urlretrieve

%matplotlib inline

## Retrieve raw data

In [ ]:
def get_amprion_wind_data(start_day, end_day):
    url = f"https://www.amprion.net/api/grid-data/items/csv/WINDDATEN/{start_day}/{end_day}"
    filename = f"wind_amprion_from_{start_day}_to_{end_day}.csv"
    
    if not os.path.isfile(filename):
        urlretrieve(url, filename)
    df = pd.read_csv(
        filename,
        delimiter=";",
        decimal=",",
        dtype={
            "Online Hochrechnung [MW]": float,
            "8:00 Uhr Prognose [MW]": float
        })
    
    return df

Retrieve raw data from Aprion Website

In [ ]:
data = get_amprion_wind_data(start_day, end_day)

Combine date and time information into a single column a set this as index

In [ ]:
data["starttime"] = data["Uhrzeit"].str.split(" - ", expand=True)[0]
data["full_date"] = data[["Datum", "starttime"]].agg(" ".join, axis=1)
data["date"] = pd.to_datetime(data["full_date"], format="%d.%m.%Y %H:%M")
data.drop(["Datum", "Uhrzeit", "starttime", "full_date"], axis=1, inplace=True)
data.set_index("date", inplace=True)

Check the column types

In [ ]:
data.dtypes

Are NaNs included in the data columns?

In [ ]:
data[data.isna().any(axis=1)]

Are there missing dates?

In [ ]:
ref_date_range = pd.date_range(data.index.min(), data.index.max(), freq="15Min")
ref_date_range[~ref_date_range.isin(data.index)]

Are the duplicate dates in the index?

In [ ]:
data[data.index.duplicated()]

Yepp! Better remove them...

In [ ]:
data = data[~data.index.duplicated()]

## Fill missing data by linear interpolation

Missing index values are inserted first with NaNs

In [ ]:
data = data.reindex(ref_date_range, fill_value=None)
data[data.isna().any(axis=1)]

Prior existing NaNs and newly inserted NaNs are replaced by linear interpolation

In [ ]:
data = data.interpolate(axis=0)
data[data.isna().any(axis=1)]

## Get a visual impression of the time series

In [ ]:
fig = px.line(
    data, 
    x=data.index, 
    #y=["Online Hochrechnung [MW]", "8:00 Uhr Prognose [MW]"],
    y=["Online Hochrechnung [MW]"],
    width=960,
)

fig.update_xaxes(rangeslider_visible=True)
fig.update_layout(showlegend=False)

### Distribution of values

In [ ]:
fig = px.histogram(data, x="Online Hochrechnung [MW]")
fig.show()

### Indentify seasonality and repeating patterns

In [ ]:
aggregations = ["Y", "M", "W", "D"]
fig = make_subplots(rows=len(aggregations), cols=1)

for row, resampler in enumerate(aggregations, start=1):
    resampled = data["Online Hochrechnung [MW]"].resample(resampler).mean()

    fig.append_trace(
        go.Scatter(
            x=resampled.index,
            y=resampled,
        ), row=row, col=1)
fig.show()

Observations

- A trend of increasing electricity generation by wind power
- Seasonality: during summer wind power generation is lower

### Do certain months have more variation?

In [ ]:
variations = data.copy()
variations["week"] = variations.index.isocalendar().week

In [ ]:
plt.figure(figsize=(16, 6))
sns.violinplot(x=variations["week"], y=variations["Online Hochrechnung [MW]"])

Observations

- Highest wind generation during winter months
- Less variation during summer
- Smaller maximum power generation during summer

# Feature engineering

## Transform time stamp

Time stamp data is transformed to separate columns that i.e. represent month of the year to reflect seasonality. This step is performed for easier interpretation of data during model training phase.

In [ ]:
data.dtypes

In [ ]:
data["year"] = data.index.year
data["month"] = data.index.month
data["week"] = data.index.isocalendar().week
data["day"] = data.index.dayofyear
data["hour"] = data.index.hour
data["quarter_hour"] = (data.index.minute.to_numpy() / 15).astype(int)

In [ ]:
data

## Reshape data and and prepare for Keras API

- **Timesteps**: define forecast length in number of timesteps. Typically, a symmetric time window is used to predict _n_ future values by data from _n_ timesteps in the past. Here, we use 144 ($=36 h \cdot 4$) in order to predict wind energy production at 12 am for the entire next day.
- **Stride**: is the number of timesteps the windows moves. Two options:

  1. As we want to predict data for the entire next day each day a 12 o'clock, we choose 96 ($=24 h \cdot 4$) as stride in order to make the prediction once a day
  2. As this is only relevant for training, we should use a stride of 1 in order to consider as much data as possbile during training


- **Batch size** describes how many samples are shown at once during training of the network. 64 is a typical value that's good to start with
- **Training set size**  is the size of the entire dataset that is used for training. Often, a split of $90\,\%$ training data and $10\,\%$ test data is used. It is important that the training set size is a multiple of batch size ($training\_set\_size \mod batch\_size \overset{!}{=} 0$)
- **Input shape:** is defined by the Keras API. It expects $input\_shape = (batches \times timesteps \times features)$

### Extract features and normalize data

Use MinMaxScaler from sklearn and explain why this one should be used here

**Hint** (for later research):
It might be useful to apply a scaler to hour, month, ... that is able to reflect cyclic characteristic of these features. Read more in [this article](http://blog.davidkaleko.com/feature-engineering-cyclical-features.html)

In [ ]:
features = data[['Online Hochrechnung [MW]', 'year', 'month', 'week', 'day', 'hour', 'quarter_hour']].to_numpy()

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from pickle import dump

scaler = MinMaxScaler(feature_range=(0, 1))
features = scaler.fit_transform(features)
features
scaler.scale_

### Training data, validation data and batch size
Split data into training and validation dataset considering the batch size

Identify exact length of training data matrix considering the constraint that training set length must be multiple of batch_size

In [ ]:
def get_training_set_length(data_length, test_fraction, batch_size):
    
    # Incrementaly increase training set length by 1
    for increment in range(0, batch_size - 1):
        train_length = round(data_length * (1 - test_fraction)) + increment
        
        # Return training set length if training set length is a multiple of batch_size
        if ((train_length - (timesteps_past - 1))%(batch_size)) == 0:
            return train_length   

In [ ]:
train_length = get_training_set_length(features.shape[0], test_fraction, batch_size)
print(train_length)
print(train_length/(timesteps_past + (stride - 1) * batch_size))

Extract training and test data from the entire dataset. Beware, test data is simply taken from the end of the entire sequence. This might bias the data.

In [ ]:
features_train = features[:train_length]
features_test = features[train_length + 1:]

### Transform data according to prediction window

Labels (produced wind energy) are extracted from features and shifted by _timesteps_future_ in order to make the model learn to predict values from past observed values.

There are two options to correlate time features with wind energy production

1. Time information is related to x **(chosen)**
2. Time information is related to y

In [ ]:
labels_start = timesteps_past + timesteps_future
labels_end = labels_start + train_length

x_train = features[:train_length]
y_train = features[labels_start:labels_end, 0]

print(x_train.shape)
print(y_train.shape)

Transform also test data

In [ ]:
total_test_length_x = len(features[train_length:(-timesteps_past - timesteps_future)])
for decrement in range(0, batch_size - 1):
    test_length_x = total_test_length_x - decrement
    
    if (test_length_x - (timesteps_past - 1))%batch_size == 0:
        break
#(test_length_x-23)/64
test_length_x

In [ ]:
x_test_end = train_length + test_length_x
test_labels_start = train_length + timesteps_past + timesteps_future
test_labels_end = train_length + labels_start + test_length_x

x_test = features[train_length:x_test_end]
y_test = features[test_labels_start:test_labels_end, 0]

print(x_test.shape)
print(y_test.shape)

## Performance metric and optimizer choice

[This article](https://otexts.com/fpp2/accuracy.html#scale-dependent-errors) gives a good overview on performance metrics for time series forecasting. Here, we choose mean absolute error (MAE) as perfomance metric (loss function) because

- It's simple to explain
- As data is scaled to the range 0..1, the unit doesn't matter

As optimizer to find best weights _Adam_ is used. It's popular optimizer and often used with LSTM for time series forecasting.

For performance evualation, mean absolute percentage error (MAPE) is calculated too, because it's easier to interpret.

Bring features and labels into shape of $(samples \times timesteps \times features)$

In [ ]:
dataset_train = keras.preprocessing.timeseries_dataset_from_array(
    x_train,
    y_train,
    sequence_length=timesteps_past,
    batch_size=batch_size,
)

dataset_test = keras.preprocessing.timeseries_dataset_from_array(
    x_test,
    y_test,
    sequence_length=timesteps_past,
    batch_size=batch_size,
)

for batch in dataset_train:
    inputs, targets = batch
    
    inputs_shape = inputs.numpy().shape
    assert inputs_shape[0] == batch_size
    assert inputs_shape[1] == timesteps_past
    assert inputs_shape[2] == 7
    assert targets.numpy().shape[0] == batch_size

print(inputs_shape, targets.numpy().shape)

In [ ]:
for batch in dataset_test: 
    inputs, targets = batch
    
    inputs_shape = inputs.numpy().shape
    assert inputs_shape[0] == batch_size
    assert inputs_shape[1] == timesteps_past
    assert inputs_shape[2] == 7
    assert targets.numpy().shape[0] == batch_size
print(inputs_shape, targets.numpy().shape)

## Defining the neurol network architecture

- Input shape: only timesteps and number features are defined
- Single-layer LSTM network with 144 units
- Dense layer with one neuron representing one feature to predict

In [ ]:
learning_rate = 0.001

input_layer = keras.layers.Input(shape=(inputs.shape[1], inputs.shape[2]), batch_size=batch_size)
lstm_2 = keras.layers.LSTM(lstm_nodes, stateful=stateful, return_sequences=False)(input_layer)
output_layer = keras.layers.Dense(1)(lstm_2)

model = keras.models.Model(inputs=input_layer, outputs=output_layer)

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=learning_rate), 
    loss=["mae"],
    metrics=[
        keras.metrics.mean_squared_error,
        keras.metrics.mean_absolute_error, #already represented by loss
        keras.metrics.mean_absolute_percentage_error],
    run_eagerly=False)
model.summary()

## Train model



**Hint:** In the Keras [timeseries forecasting example](https://keras.io/examples/timeseries/timeseries_weather_forecasting/#training) model checkpoints and early stopping, based on validation loss is used. Consider to follow this approach for faster training, once validation data is available

In [ ]:
if train_model:
    history = model.fit(
        dataset_train,
        epochs=epochs,
        shuffle=False,
    )
    model.save(filename_model)
    print(history.history)
else:
    model = keras.models.load_model(filename_model)

## Evaluate model performance

In [ ]:
model.evaluate(dataset_test)

In [ ]:
predictions = []
true_labels = []

for num, (x, y) in enumerate(dataset_test): 
    
    true_labels.extend(y.numpy())

    prediction = model.predict(x)
    predictions.extend(prediction)

predictions = np.array(predictions).flatten()
prediction_evaluation = pd.DataFrame(
    {
        "predictions": predictions,
        "labels": true_labels
    })

### Scale data back to original unit

scale back to original data range of features

In [ ]:
def inverse_transform(series, n_features):

    trainPredict_dataset_like = np.zeros(shape=(len(series), n_features))
    trainPredict_dataset_like[:, 0] = series.to_numpy()
    return scaler.inverse_transform(trainPredict_dataset_like)[:, 0]

In [ ]:
prediction_evaluation["labels_MWh"] = inverse_transform(prediction_evaluation["labels"], 7)
prediction_evaluation["predictions_MWh"] = inverse_transform(prediction_evaluation["predictions"], 7)

In [ ]:
fig = px.line(
    prediction_evaluation, 
    x=range(len(predictions)), 
    y=["labels_MWh", "predictions_MWh"],
    width=960,
)

fig.update_xaxes(rangeslider_visible=True)
fig.update_layout(showlegend=False)

### Calculate performance indicators

Re-calculate performance indicators manually

- **Mean absolute error** as: $MAE=\frac{1}{n} \cdot \sum_{t=1}^{n} \left| y_{t}-\hat{y}_t \right|$
- **Mean absolute percentage error** as: $MAPE=\frac{100}{n} \cdot \sum_{t=1}^{n} \left| \frac{y_{t}-\hat{y}_t}{y_{t}} \right|$

But first, per row, hence, for each time step in order to explore the distribution of deviations.

Seems like to match quite well, but it might be not matching in time! Predictions seem to be lagging and like predicting the value of the last timestep

In [ ]:
prediction_evaluation["difference_absolute"] = (prediction_evaluation["labels_MWh"] - prediction_evaluation["predictions_MWh"]).abs()
prediction_evaluation["difference_percentage"] = (prediction_evaluation["difference_absolute"] / prediction_evaluation["labels_MWh"]) * 100
prediction_evaluation.loc[prediction_evaluation["difference_percentage"] == np.inf, "difference_percentage"] = 0

In [ ]:
px.histogram(prediction_evaluation, x=["difference_percentage"])

In [ ]:
print(f"Mean absolute error (MAE): {prediction_evaluation['difference_absolute'].mean()}")
print(f"Mean absolute percentage error (MAPE): {prediction_evaluation['difference_percentage'].mean()}")